# Lecture 03 - Multiple Linear Regression

## Import Required Libraries

In [ ]:
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
!pip install dmba

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 70.1 MB/s eta 0:00:00
Colab environment detected.


## Load the Data

In [ ]:
my_drive_path = "/content/drive/MyDrive/SUNY/Class Material/2024 Fall/MSA550A/Python Class Work/msa550-code-files/data/"

In [ ]:
car_df = pd.read_csv(my_drive_path+'ToyotaCorolla.csv')
print(car_df.head(5))
car_df.columns

   Id                                          Model  Price  Age_08_04  \
0   1  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13500         23   
1   2  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13750         23   
2   3  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13950         24   
3   4  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  14950         26   
4   5    TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors  13750         30   

   Mfg_Month  Mfg_Year     KM Fuel_Type  HP  Met_Color  ... Powered_Windows  \
0         10      2002  46986    Diesel  90          1  ...               1   
1         10      2002  72937    Diesel  90          1  ...               0   
2          9      2002  41711    Diesel  90          1  ...               0   
3          7      2002  48000    Diesel  90          0  ...               0   
4          3      2002  38500    Diesel  90          0  ...               1   

   Power_Steering  Radio  Mistlamps  Sport_Model  Backseat_Divider  \
0         

Index(['Id', 'Model', 'Price', 'Age_08_04', 'Mfg_Month', 'Mfg_Year', 'KM',
       'Fuel_Type', 'HP', 'Met_Color', 'Color', 'Automatic', 'CC', 'Doors',
       'Cylinders', 'Gears', 'Quarterly_Tax', 'Weight', 'Mfr_Guarantee',
       'BOVAG_Guarantee', 'Guarantee_Period', 'ABS', 'Airbag_1', 'Airbag_2',
       'Airco', 'Automatic_airco', 'Boardcomputer', 'CD_Player',
       'Central_Lock', 'Powered_Windows', 'Power_Steering', 'Radio',
       'Mistlamps', 'Sport_Model', 'Backseat_Divider', 'Metallic_Rim',
       'Radio_cassette', 'Parking_Assistant', 'Tow_Bar'],
      dtype='object')

## Preprocessing the Data

### Handling Missing Values
Remember, the fact that it has no null values, does not mean no missing values.

In [ ]:
print("Missing values:\n", car_df.isnull().sum())

Missing values:
 Id                   0
Model                0
Price                0
Age_08_04            0
Mfg_Month            0
Mfg_Year             0
KM                   0
Fuel_Type            0
HP                   0
Met_Color            0
Color                0
Automatic            0
CC                   0
Doors                0
Cylinders            0
Gears                0
Quarterly_Tax        0
Weight               0
Mfr_Guarantee        0
BOVAG_Guarantee      0
Guarantee_Period     0
ABS                  0
Airbag_1             0
Airbag_2             0
Airco                0
Automatic_airco      0
Boardcomputer        0
CD_Player            0
Central_Lock         0
Powered_Windows      0
Power_Steering       0
Radio                0
Mistlamps            0
Sport_Model          0
Backseat_Divider     0
Metallic_Rim         0
Radio_cassette       0
Parking_Assistant    0
Tow_Bar              0
dtype: int64


### Correlation Table
Multicollinearity check, variable relationship etc.

In [ ]:
numeric_columns = car_df.select_dtypes(include=[np.number]).columns

# Create a new DataFrame with only numeric columns
numeric_df = car_df[numeric_columns]

correlation_matrix = numeric_df.corr()
#print(correlation_matrix)

fig = go.Figure(data=go.Heatmap(
                z=correlation_matrix.values,
                x=correlation_matrix.columns,
                y=correlation_matrix.index,
                colorscale='RdBu',
                zmin=-1, zmax=1,
                text=correlation_matrix.values,
                texttemplate='%{text:.2f}',
                hoverongaps=False))

# Update the layout
fig.update_layout(
    title='Correlation Matrix',
    xaxis_title='Features',
    yaxis_title='Features',
    width=800,  # Adjust the width as needed
    height=700  # Adjust the height as needed
)

# Show the plot
fig.show()

1. Strong correlations with Price:
* Age_08_04 has a strong negative correlation (-0.876590) with Price, indicating that older cars tend to be cheaper.
* Mfg_Year has a strong positive correlation (0.885159) with Price, which is consistent with the Age correlation.
* KM (kilometers) has a moderate negative correlation (-0.569960) with Price, suggesting that cars with higher mileage tend to be cheaper.

2. Multicollinearity:
* Age_08_04 and Mfg_Year are highly correlated (-0.983661), which is expected as they represent similar information. **drop Mfg_Year**
* Id seems to be correlated with several variables, which is unusual and might indicate it's not just a simple identifier. **drop Id**

3. Lack of correlation:
* Some features like Radio, Power_Steering, and Metallic_Rim show very weak correlations with Price. **will drop them**

4. Potential redundant features:
* Radio and Radio_cassette are highly correlated (0.991621), suggesting they might be redundant. *drop them - (radio was also mentioned in point 3)**

In [ ]:
# for Radio_cassette, Radio, Power_Steering, and Metallic_Rim, Mfg_Year Id we planned to drop them already

#in addition: CD_Player, Backseat_Divider, BOVAG_Guarantee, Guarantee_Period, ABS, Automatic_airco

columns_to_drop = ['Radio_cassette', 'Radio', 'Power_Steering', 'Metallic_Rim', 'Mfg_Year', 'Id','CD_Player','Backseat_Divider','BOVAG_Guarantee','Guarantee_Period','Automatic_airco','ABS']

car_df = car_df.drop(columns=columns_to_drop)

print("Columns dropped successfully.")
print(f"Remaining columns: {car_df.columns.tolist()}")

Columns dropped successfully.
Remaining columns: ['Model', 'Price', 'Age_08_04', 'Mfg_Month', 'KM', 'Fuel_Type', 'HP', 'Met_Color', 'Color', 'Automatic', 'CC', 'Doors', 'Cylinders', 'Gears', 'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'Airbag_1', 'Airbag_2', 'Airco', 'Boardcomputer', 'Central_Lock', 'Powered_Windows', 'Mistlamps', 'Sport_Model', 'Parking_Assistant', 'Tow_Bar']


### Check for Outliers

In [ ]:
# 4. Check for outliers
numeric_columns = car_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create subplots, one for each numeric column
fig = make_subplots(rows=1, cols=len(numeric_columns),
                    subplot_titles=numeric_columns)

# Add box plots for each numeric column
for i, col in enumerate(numeric_columns):
    fig.add_trace(
        go.Box(y=car_df[col], name=col),
        row=1, col=i+1
    )

# Update layout
fig.update_layout(
    title_text="Boxplots to Check for Outliers",
    height=600,
    width=200 * len(numeric_columns),  # Adjust width based on number of plots
    showlegend=False
)

# Update x-axis
fig.update_xaxes(visible=False)  # Hide x-axis labels as they're not needed for boxplots

# Show plot
fig.show()

In [ ]:
#outlier stats
numeric_columns = car_df.select_dtypes(include=['int64', 'float64']).columns

outlier_percentages = {}

# Print summary statistics for each numeric column
for column in numeric_columns:
    print(f"\nSummary statistics for {column}:")
    print(car_df[column].describe())
    print("\nPotential outliers:")
    Q1 = car_df[column].quantile(0.25)
    Q3 = car_df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = car_df[(car_df[column] < lower_bound) | (car_df[column] > upper_bound)][column]
    print(outliers)

    # Calculate percentage of outliers
    outlier_percentage = (len(outliers) / len(car_df)) * 100
    outlier_percentages[column] = outlier_percentage

    print(f"Percentage of outliers: {outlier_percentage:.2f}%")
    print("--------------------")

# Print total percentage of outliers for each parameter
print("\nTotal percentage of outliers for each parameter:")
for column, percentage in outlier_percentages.items():
    print(f"{column}: {percentage:.2f}%")


Summary statistics for Price:
count     1436.000000
mean     10730.824513
std       3626.964585
min       4350.000000
25%       8450.000000
50%       9900.000000
75%      11950.000000
max      32500.000000
Name: Price, dtype: float64

Potential outliers:
7      18600
8      21500
10     20950
11     19950
12     19600
       ...  
182    21125
183    21500
184    17795
185    18245
523    18950
Name: Price, Length: 110, dtype: int64
Percentage of outliers: 7.66%
--------------------

Summary statistics for Age_08_04:
count    1436.000000
mean       55.947075
std        18.599988
min         1.000000
25%        44.000000
50%        61.000000
75%        70.000000
max        80.000000
Name: Age_08_04, dtype: float64

Potential outliers:
109    4
110    4
111    4
182    2
183    2
184    1
185    1
Name: Age_08_04, dtype: int64
Percentage of outliers: 0.49%
--------------------

Summary statistics for Mfg_Month:
count    1436.000000
mean        5.548747
std         3.354085
min         1

In [ ]:
def remove_outliers(df, columns):
    df_clean = df.copy()
    total_removed = 0
    total_data = len(df) * len(columns)

    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        df_clean = df_clean[~df_clean.index.isin(outliers.index)]

        num_removed = len(outliers)
        total_removed += num_removed

        print(f"Column {column}:")
        print(f"  Number of outliers removed: {num_removed}")
        print(f"  Percentage of data removed: {(num_removed / len(df)) * 100:.2f}%")
        print("--------------------")

    overall_percentage = (total_removed / total_data) * 100
    print(f"\nOverall percentage of data points removed: {overall_percentage:.2f}%")
    print(f"Number of rows in original dataset: {len(df)}")
    print(f"Number of rows in cleaned dataset: {len(df_clean)}")

    return df_clean

print(car_df.shape)
# Remove outliers - be careful with this one!!!
car_df_clean = remove_outliers(car_df, numeric_columns)
car_df_clean.shape

(1436, 27)
Column Price:
  Number of outliers removed: 110
  Percentage of data removed: 7.66%
--------------------
Column Age_08_04:
  Number of outliers removed: 7
  Percentage of data removed: 0.49%
--------------------
Column Mfg_Month:
  Number of outliers removed: 0
  Percentage of data removed: 0.00%
--------------------
Column KM:
  Number of outliers removed: 49
  Percentage of data removed: 3.41%
--------------------
Column HP:
  Number of outliers removed: 11
  Percentage of data removed: 0.77%
--------------------
Column Met_Color:
  Number of outliers removed: 0
  Percentage of data removed: 0.00%
--------------------
Column Automatic:
  Number of outliers removed: 80
  Percentage of data removed: 5.57%
--------------------
Column CC:
  Number of outliers removed: 123
  Percentage of data removed: 8.57%
--------------------
Column Doors:
  Number of outliers removed: 0
  Percentage of data removed: 0.00%
--------------------
Column Cylinders:
  Number of outliers removed: 

(978, 27)

### Encode Categorical Variables

In [ ]:
# 5. Encode categorical variables
car_df_model = pd.get_dummies(car_df, columns=['Fuel_Type'], drop_first=True)

In [ ]:
car_df_model.columns

Index(['Model', 'Price', 'Age_08_04', 'Mfg_Month', 'KM', 'HP', 'Met_Color',
       'Color', 'Automatic', 'CC', 'Doors', 'Cylinders', 'Gears',
       'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'Airbag_1', 'Airbag_2',
       'Airco', 'Boardcomputer', 'Central_Lock', 'Powered_Windows',
       'Mistlamps', 'Sport_Model', 'Parking_Assistant', 'Tow_Bar',
       'Fuel_Type_Diesel', 'Fuel_Type_Petrol'],
      dtype='object')

### Scale Numerical Variables


In [ ]:
scaler = StandardScaler()
#hand picked them
numerical_columns = ['Age_08_04', 'KM', 'HP', 'CC', 'Weight']
car_df_model[numerical_columns] = scaler.fit_transform(car_df[numerical_columns])

### Feature Engineering and Transformations
You can create new features to better represent data.
You can transform variables as well

In [ ]:
# Feature engineering (example)  -- It captures the relationship between price and mileage in a single feature
car_df['Price_per_KM'] = car_df['Price'] / (car_df['KM'] + 1)  # Adding 1 to avoid division by zero


# Logarithmic transformation of Price - It can help normalize the distribution of prices if they are right-skewed
car_df['Log_Price'] = np.log(car_df['Price']) #In economic models, log-transformations are often used because many economic phenomena follow log-normal distributions.


In [ ]:
# Additional: Check VIF for multicollinearity
def calculate_vif(X):
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

print("VIF:\n", calculate_vif(numeric_df))

VIF:
              Variable        VIF
0                  Id  13.780509
1               Price  10.921213
2           Age_08_04        inf
3           Mfg_Month        inf
4            Mfg_Year        inf
5                  KM   3.175433
6                  HP   1.741414
7           Met_Color   1.142551
8           Automatic   1.115682
9                  CC   1.222698
10              Doors   1.261459
11          Cylinders   0.000000
12              Gears   1.282043
13      Quarterly_Tax   2.865925
14             Weight   3.459321
15      Mfr_Guarantee   1.228304
16    BOVAG_Guarantee   1.393054
17   Guarantee_Period   1.593722
18                ABS   2.341555
19           Airbag_1   1.600533
20           Airbag_2   3.091520
21              Airco   1.849285
22    Automatic_airco   2.028305
23      Boardcomputer   2.648891
24          CD_Player   1.556553
25       Central_Lock   4.609259
26    Powered_Windows   4.665527
27     Power_Steering   1.560597
28              Radio  62.408604
29  

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning:

divide by zero encountered in scalar divide

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide



## Split the Data

In [ ]:
car_df_model.columns

Index(['Model', 'Price', 'Age_08_04', 'Mfg_Month', 'KM', 'HP', 'Met_Color',
       'Color', 'Automatic', 'CC', 'Doors', 'Cylinders', 'Gears',
       'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'Airbag_1', 'Airbag_2',
       'Airco', 'Boardcomputer', 'Central_Lock', 'Powered_Windows',
       'Mistlamps', 'Sport_Model', 'Parking_Assistant', 'Tow_Bar',
       'Fuel_Type_Diesel', 'Fuel_Type_Petrol'],
      dtype='object')

In [ ]:
# 'Age_08_04', 'KM', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'HP', 'Met_Color', 'Automatic', 'CC', 'Doors', 'Weight'

predictors = ['Age_08_04', 'KM', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'HP', 'Met_Color', 'Automatic', 'CC', 'Doors', 'Quarterly_Tax', 'Weight']
outcome = 'Price'

#select only top 1000 records
car_df_model = car_df_model.head(1000)



X = car_df_model[predictors]
y = car_df_model[outcome]





train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=42)

## Build the Model

In [ ]:
car_lm = LinearRegression()
car_lm.fit(train_X, train_y)

# print coefficients
print('intercept ', car_lm.intercept_)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': car_lm.coef_}))

# print performance measures
regressionSummary(train_y, car_lm.predict(train_X))

intercept  7549.892823319728
           Predictor  coefficient
0          Age_08_04 -2450.525317
1                 KM  -691.330378
2   Fuel_Type_Diesel  -390.607449
3   Fuel_Type_Petrol  2300.137898
4                 HP   416.241441
5          Met_Color    91.168314
6          Automatic    64.200590
7                 CC    -3.643264
8              Doors  -123.435768
9      Quarterly_Tax    17.046365
10            Weight  1104.017001

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 1240.1195
            Mean Absolute Error (MAE) : 960.6568
          Mean Percentage Error (MPE) : -0.8471
Mean Absolute Percentage Error (MAPE) : 8.6619


Component	Coefficient	Explanation
* Intercept	6619.77	The baseline value when all predictors are zero
* Age_08_04	-2307.65	For each unit increase in age, price decreases by 2,307.65
* KM	-602.10	For each additional kilometer, price decreases by 602.10
* Fuel_Type_Diesel	4481.09	Diesel cars are priced 4,481.09 higher than the baseline fuel type
* Fuel_Type_Petrol	2413.06	Petrol cars are priced 2,413.06 higher than the baseline fuel type
* HP	1131.41	Each additional horsepower increases price by 1,131.41
* Met_Color	47.72	Metallic color adds 47.72 to the price
* Automatic	462.44	Automatic transmission adds 462.44 to the price
* CC	-2132.90	Each unit increase in engine size decreases price by 2,132.90
* Doors	58.42	Each additional door adds 58.42 to the price
* Quarterly_Tax	13.01	Each unit increase in quarterly tax adds 13.01 to the price
* Weight	744.94	Each unit increase in weight adds 744.94 to the price

Key Insights:
* Age and mileage (KM) have negative impacts on price, as expected for used cars.
* Diesel fuel type has the strongest positive effect on price.
* Horsepower (HP) and weight positively influence price, indicating a premium for powerful and larger vehicles.
* Engine size (CC) surprisingly has a negative correlation, possibly due to fuel efficiency concerns.
* Automatic transmission adds value to the car.
* The number of doors and metallic color have minor positive impacts on price.

Statistic	Value
* Mean Error (ME)	-0.0000
* Root Mean Squared Error (RMSE)	1315.5318
* Mean Absolute Error (MAE)	953.7443
* Mean Percentage Error (MPE)	-1.0544
* Mean Absolute Percentage Error (MAPE)	9.2370

Interpretation:
* ME: The average of all errors is essentially zero, indicating unbiased predictions.
* RMSE: On average, predictions deviate from actual values by about 1,316 units.
* MAE: The average absolute difference between predicted and actual values is about 954 units.
* MPE: On average, predictions underestimate actual values by about 1.05%.
* MAPE: The average absolute percentage error is 9.24%, suggesting reasonable accuracy.

Key Takeaways:
* The model shows no systematic bias (ME ≈ 0).
* Predictions are generally within 9.24% of actual values.
* There's room for improvement, but the model provides useful estimates.